# Assignment 3

**Exercise**


In this exercise, you will use the ADNI dataset from the past lesson.

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.stats import norm


data = pd.read_csv('/content/adni_data')
pd.set_option('mode.chained_assignment', None) 

data_ct_ad = data.query('DX == 1 | DX == 3') 
data_ct_ad['DX'] = data_ct_ad['DX'].map({1: 0, 3: 1})

data_ct_ad['norm_brain'] = data_ct_ad['WholeBrain.bl']/data_ct_ad['ICV']
data_ct_ad['norm_brain'] = (data_ct_ad['norm_brain'] - np.mean(data_ct_ad['norm_brain']))/np.std(data_ct_ad['norm_brain'])
data_ct_ad.dropna(inplace = True)


print(data_ct_ad[:5])

   RID  APOE4  DX   AGE  WholeBrain.bl           ICV  norm_brain
0    2    0.0   0  74.3      1229738.0  1.984657e+06   -0.907716
1    3    1.0   1  81.3      1129834.0  1.920691e+06   -1.508346
3    5    0.0   0  73.7      1116633.0  1.640766e+06    0.258629
5    7    1.0   1  75.4       875798.0  1.353519e+06   -0.382669
6    8    0.0   0  84.5       948684.0  1.396068e+06    0.239182


###1) Fit a model to predict the diagnosis (DX) of the subjects using both AGE and APOE4 as predictors.**

In [48]:
#!pip install nest_asyncio

In [49]:
import stan
import nest_asyncio as asyncio
import nest_asyncio
nest_asyncio.apply()

In [50]:
binomial_model_code_2 = """
data {
  int<lower=1> N;
  int y[N];
  real x[N];
  real x_g[N];
}
parameters {
  real a;
  real b;
  real c;
}
transformed parameters {
  vector[N] p_i;
  for (i in 1:N) {
    p_i[i] = exp(a + b * x[i] + c * x_g[i])/(1 + exp(a + b * x[i] + c * x_g[i])); 
    }
}
model {
  c ~ normal(0, 3);
  b ~ normal(0, 3);
  a ~ normal(0, 3);
  y ~ binomial(1, p_i);
}
"""

In [51]:
data_to_stan2 = dict(x = np.array(data_ct_ad['AGE']), 
                    x_g = np.array(data_ct_ad['APOE4']),
                    y = np.array(data_ct_ad['DX']), 
                    N = len(data_ct_ad['DX']))

seed = 123

posterior2 = stan.build(binomial_model_code_2, data=data_to_stan2, random_seed=seed)

Building...



Building: 21.6s, done.

In [52]:
fit2 = posterior2.sample(num_chains=4, num_samples=1000)
df2 = fit2.to_frame()
print(df2.describe().T[:10])

Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   1% (101/8000)
Sampling:   3% (201/8000)
Sampling:   4% (301/8000)
Sampling:   5% (401/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  18% (1400/8000)
Sampling:  19% (1501/8000)
Sampling:  20% (1601/8000)
Sampling:  21% (1701/8000)
Sampling:  22% (1800/8000)
Sampling:  24% (1900/8000)
Sampling:  25% (2000/8000)
Sampling:  26% (2100/8000)
Sampling:  28% (2200/8000)
Sampling:  29% (2301/8000)
Sampling:  30% (2401/8000)
Sampling:  31% (2500/8000)
Sampling:  32% (2600/8000)
Sampling:  34% (2700/8000)
Sampling:  35% (2800/8000)
Sampling:  36% (2900/8000)
Sampling:  38% (3000/8000)
Sampling:  39% (3100/8000)
Sampling:  40% (3200/8000)
Sampling:  41% (3300/8000)
Sampling:  42% (3400/8000)
Sampling:  44% (3500/8000)
S

                count        mean        std         min         25%  \
parameters                                                             
lp__           4000.0 -475.328419   1.160519 -483.247067 -475.873176   
accept_stat__  4000.0    0.938920   0.087522    0.223955    0.916486   
stepsize__     4000.0    0.064550   0.004809    0.057918    0.060992   
treedepth__    4000.0    4.423500   1.422376    1.000000    4.000000   
n_leapfrog__   4000.0   38.688500  25.113694    1.000000   15.000000   
divergent__    4000.0    0.000000   0.000000    0.000000    0.000000   
energy__       4000.0  476.809436   1.676744  473.910928  475.581167   
a              4000.0   -3.947208   0.876653   -6.941313   -4.581999   
b              4000.0    0.035796   0.011483    0.002105    0.027849   
c              4000.0    1.477997   0.129367    1.033879    1.386660   

                      50%         75%         max  
parameters                                         
lp__          -475.036980 -474.

In [53]:
p_i2 = fit2['p_i']

lppd2 = []
pwaic2 = []
for i in range(len(data_ct_ad)):
    id_log_lik2 = []
    for k in range(1000):
        p = p_i2[i,k]
        id_log_lik2.append(scipy.stats.binom.logpmf(data_ct_ad['DX'].values[i], 1, p))
    lppd2.append(scipy.special.logsumexp(id_log_lik2) - np.log(len(id_log_lik2)))
    pwaic2.append(np.var(id_log_lik2))

waic_apoe_age = -2 * (np.sum(lppd2) -np.sum(pwaic2))

In [54]:
print(waic_apoe_age)

951.6944483594174


We got the WAIC score 951.69 when we use both AGE and APOE4 as predictors.

###2) Consider subjects who are 80 years old and check the effect of the APOE4 gene on the diagnosis.**

Hint: You'll draw many samples from two binomial distributions. One where APOE4 is included in the computation of $p_i$ and one where it's not.

Firstly, we extract the patients whose age is 80

In [55]:
data_age80 = data_ct_ad.query("AGE == 80")
data_age80

,RID,APOE4,DX,AGE,WholeBrain.bl,ICV,norm_brain
139,230,0.0,0,80.0,1051053.0,1.714028e+06,-1.030522
489,866,0.0,0,80.0,943825.0,1.388961e+06,0.238778
522,920,1.0,0,80.0,946606.0,1.464818e+06,-0.398452
740,1285,1.0,1,80.0,1025968.0,1.626103e+06,-0.691147


There are only 4 patients with age 80.


The goal in this question is to determine the effect of APOE4 on the Alzheimer disease. We will draw samples from two binomial distributions:

1. One binomial distribution that does not include  APOE4  in the computation of  𝑝𝑖  
2. One binomial distribution that does include  APOE4  in the computation of  𝑝𝑖 

####2.1 The binomial distribution that does NOT include APOE4 in the computation of 𝑝𝑖

  In this model we will only fit `DX` with an intercept $a$~$N(0,3)$:

\begin{align}
p_i &= \frac{\exp(a)}{1 + \exp(a)}
\end{align}



In [75]:
age80_not_include_APOE4 = """
data {
  int<lower=1> N;
  int y[N];
}
parameters {
  real a;
}
transformed parameters {
  vector[N] p_i;
  for (i in 1:N) {
    p_i[i] = exp(a)/(1 + exp(a)); 
    }
}
model {
  a ~ normal(0, 10);
  y ~ binomial(1, p_i);
}
"""

(There will be no x in the model and in the data

In [76]:
data_age80_not_include_APOE4 = dict(y = np.array(data_age80['DX']), 
                                    N = len(data_age80['DX']))

seed = 123

posterior_age80_not_include_APOE4 = stan.build(age80_not_include_APOE4, data=data_age80_not_include_APOE4, random_seed=seed)

Building...



Building: 20.6s, done.

In [77]:
fit3 = posterior_age80_not_include_APOE4.sample(num_chains=4, num_samples=1000)
df3 = fit3.to_frame()
#print(df3.describe().T[:10])

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: binomial_lpmf: Probability parameter[1] is -nan, but must be in the interval [0, 1] (in '/tmp/httpstan_modxi9e4/model_xb3dkc65.stan', line 17, column 2 to column 23)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model 

In [78]:
p_i3 = fit3['p_i']

lppd3 = []
pwaic3 = []
for i in range(len(data_age80)):
    id_log_lik3 = []
    for k in range(1000):
        p = p_i3[i,k]
        id_log_lik3.append(scipy.stats.binom.logpmf(data_age80['DX'].values[i], 1, p))
    lppd3.append(scipy.special.logsumexp(id_log_lik3) - np.log(len(id_log_lik3)))
    pwaic3.append(np.var(id_log_lik3))

waic_age80_not_include_APOE4 = -2 * (np.sum(lppd3) -np.sum(pwaic3))

In [79]:
print(waic_age80_not_include_APOE4)

7.947906955902098


####2.2 The binomial distribution that does include APOE4 in the computation of 𝑝𝑖

In this case we will use the model described by a prior normal distribution as we did in the question 1

In [80]:
age80_include_APOE4 = """
data {
  int N;
  int y[N];
  real x[N];
}
parameters {
  real a;
  real b;
}
transformed parameters {
  vector[N] p_i;
  for (i in 1:N) {
    p_i[i] = exp(a + b * x[i])/(1 + exp(a + b * x[i])); 
    }
}
model {
  a ~ normal(0, 10);
  b ~ normal(0, 10); 
  y ~ binomial(1, p_i);
}
"""

In [81]:
data_age80_include_APOE4 = dict(x = np.array(data_age80['APOE4']),
                                y = np.array(data_age80['DX']), 
                                N = len(data_age80['DX']))

seed = 123

posterior_age80_include_APOE4 = stan.build(age80_include_APOE4, data=data_age80_include_APOE4, random_seed=seed)

Building...



Building: 21.1s, done.

In [82]:
fit4 = posterior_age80_include_APOE4.sample(num_chains=4, num_samples=1000)
df4 = fit4.to_frame()
#print(df4.describe().T[:10])

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.04 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!


In [83]:
p_i4 = fit4['p_i']

lppd4 = []
pwaic4 = []
for i in range(len(data_age80)):
    id_log_lik4 = []
    for k in range(1000):
        p = p_i4[i,k]
        id_log_lik4.append(scipy.stats.binom.logpmf(data_age80['DX'].values[i], 1, p))
    lppd4.append(scipy.special.logsumexp(id_log_lik4) - np.log(len(id_log_lik4)))
    pwaic4.append(np.var(id_log_lik4))

waic_age80_include_APOE4 = -2 * (np.sum(lppd4) -np.sum(pwaic4))

In [84]:
print(waic_age80_include_APOE4)

6.867027604796464


So now let's print both of the results and compare them:

In [86]:
print("WAIC score without APOE4 =", waic_age80_not_include_APOE4)
print("WAIC score wit APOE4 =", waic_age80_include_APOE4)

WAIC score without APOE4 = 7.947906955902098
WAIC score wit APOE4 = 6.867027604796464


The WAIC score decreased when we use APOE4, that means adding APOE4 positively impacts the accuracy of a prediction of the disease in patients whose age are 80.

###3) In the last lesson, we fitted a model to predict the diagnosis using only the size of the brain (norm_brain). Compare this model and the one of question 1 in terms of WAIC. Is one better than the other ?**

Firstly, I will fit the model to predict the diagnosis using only the size of the brain (norm_brain) and get the WAIC result. Then I will compare the result with the WAIC score in Question-1.

We write the Stan code with only one feature(norm_brain):

In [67]:
binomial_model_code = """
data {
  int<lower=1> N;
  int y[N];
  real x[N];
}
parameters {
  real a;
  real b;
}
transformed parameters {
  vector[N] p_i;
  for (i in 1:N) {
    p_i[i] = exp(a + b * x[i])/(1 + exp(a + b * x[i])); 
    }
}
model {
  b ~ normal(0, 3);
  a ~ normal(0, 3);
  y ~ binomial(1, p_i);
}
"""


We define the x and y. Then we find the posterior 

In [68]:
data_to_stan = dict(x = np.array(data_ct_ad['norm_brain']), y = np.array(data_ct_ad['DX']), N = len(data_ct_ad['DX']))

seed = 123

posterior = stan.build(binomial_model_code, data=data_to_stan, random_seed=seed)

Building...



Building: 21.1s, done.

We fit the model:

In [69]:
fit = posterior.sample(num_chains=4, num_samples=1000)
df = fit.to_frame()
print(df.T[:10])

Sampling:   0%
Sampling:   1% (100/8000)
Sampling:   2% (200/8000)
Sampling:   6% (500/8000)
Sampling:  10% (800/8000)
Sampling:  14% (1100/8000)
Sampling:  18% (1400/8000)
Sampling:  34% (2700/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 0.000331 seconds
  1000 transitions using 10 leapfrog steps per transition would take 3.31 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: binomial_lpmf: Probability parameter[1] is -nan, but must be in the interval [0, 1] (in '/tmp/httpstan_mloao_hw/model_hzwpgw3l.stan', line 20, column 2 to column 23)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may

draws                0           1           2           3           4     \
parameters                                                                  
lp__          -459.482863 -459.641495 -460.052334 -459.391449 -459.581164   
accept_stat__    0.929982    0.863033    0.993246    0.995092    0.971044   
stepsize__       0.815036    0.859751    0.775350    0.819602    0.815036   
treedepth__      2.000000    2.000000    3.000000    2.000000    2.000000   
n_leapfrog__     3.000000    3.000000    7.000000    7.000000    3.000000   
divergent__      0.000000    0.000000    0.000000    0.000000    0.000000   
energy__       461.400840  462.655082  460.370410  459.624451  459.869255   
a               -0.593770   -0.511488   -0.573679   -0.535464   -0.575550   
b               -1.212961   -1.159228   -1.024701   -1.125972   -1.067033   
p_i.1            0.624162    0.631989    0.588183    0.619305    0.597008   

draws                5           6           7           8           9     

In [70]:
est_a = fit['a'][0]
quantiles_a = np.quantile(est_a, [0.05, 0.5, 0.95])

est_b = fit['b'][0]
quantiles_b = np.quantile(est_b, [0.05, 0.5, 0.95])

logistic_a_50 = np.exp(quantiles_a[1])/(1 + np.exp(quantiles_a[1]))
logistic_a_5 = np.exp(quantiles_a[0])/(1 + np.exp(quantiles_a[0]))
logistic_a_95 = np.exp(quantiles_a[2])/(1 + np.exp(quantiles_a[2]))

print('Baseline probability of disease: \n')
print('5%  :', logistic_a_5)
print('50% :', logistic_a_50)
print('95% :', logistic_a_95)


logistic_b_95 = np.exp(quantiles_a[0] - quantiles_b[0])/(1 + np.exp(quantiles_a[0] - quantiles_b[0]))
logistic_b_50 = np.exp(quantiles_a[0] - quantiles_b[1])/(1 + np.exp(quantiles_a[0] - quantiles_b[1]))
logistic_b_5 = np.exp(quantiles_a[0] - quantiles_b[2])/(1 + np.exp(quantiles_a[0] - quantiles_b[2]))


print('Probability increase for unit decrease in standardized brain volume (parameter "a" at lowest quantile): \n')
print('5%  :', logistic_b_5)
print('50% :', logistic_b_50)
print('95% :', logistic_b_95)

Baseline probability of disease: 

5%  : 0.3279238468020261
50% : 0.35850834596145853
95% : 0.3904052861126189
Probability increase for unit decrease in standardized brain volume (parameter "a" at lowest quantile): 

5%  : 0.5678414993276841
50% : 0.6062759855194331
95% : 0.6468168430959407


In [71]:
p_i = fit['p_i']
print(p_i.shape)

(826, 4000)


In [72]:

lppd = []
pwaic = []
for i in range(len(data_ct_ad)):
    id_log_lik = []
    for k in range(1000):
        p = p_i[i,k]
        id_log_lik.append(scipy.stats.binom.logpmf(data_ct_ad['DX'].values[i], 1, p))
    lppd.append(scipy.special.logsumexp(id_log_lik) - np.log(len(id_log_lik)))
    pwaic.append(np.var(id_log_lik))

waic_norm_brain = -2 * (np.sum(lppd) -np.sum(pwaic))

Let's compare the WAIC using APOE4 and AGE with the WAIC using norm_brain.

In [73]:
print("WAIC score using only norm_brain = ", waic_norm_brain)
print("WAIC score using APOE4 and AGE = ", waic_apoe_age)

WAIC score using only norm_brain =  922.6726756928985
WAIC score using APOE4 and AGE =  951.6944483594174


The WAIC score using only norm_brain is less than the other. The less is better in the WAIC score comparison. We can conclude that using norm_brain alone is a better classification feature than using the APOE4 and AGE together.